In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs
import time
from arbol_n_dimensiones import Arbol_n_dimensiones

In [2]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) #lo dejo en metros

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [3]:
propiedades = pd.read_csv('/home/mati/Desktop/properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [4]:
propiedades = propiedades.loc[propiedades.Year >= 2017,:]

In [5]:
propiedades = propiedades.loc[(propiedades.lat.notnull()) & (propiedades.lon.notnull()) &\
                             (propiedades.superficie.notnull()) & (propiedades.price_aprox_usd.notnull()),:]
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294660 entries, 1988104 to 2455308
Data columns (total 23 columns):
created_on                    294660 non-null object
currency                      293968 non-null object
expenses                      41414 non-null float64
fecha_de_publicacion          294660 non-null object
floor                         36468 non-null float64
lat                           294660 non-null float64
lon                           294660 non-null float64
place_name                    294596 non-null object
place_with_parent_names       294660 non-null object
price                         294660 non-null float64
price_aprox_local_currency    294660 non-null float64
price_aprox_usd               294660 non-null float64
price_per_m2                  282871 non-null float64
price_usd_per_m2              294660 non-null float64
property_type                 294660 non-null object
rooms                         198414 non-null float64
state_name                

## Tomo como set de prueba las publicaciones de junio 2017

In [6]:
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:].reset_index()
set_datos = propiedades.loc[(propiedades.Year != 2017) | (propiedades.Month < 6),:].reset_index()

In [7]:
set_pruebas = set_pruebas.sample(14166).reset_index()
set_pruebas = set_pruebas.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_pruebas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 5 columns):
property_type      14166 non-null object
lat                14166 non-null float64
lon                14166 non-null float64
superficie         14166 non-null float64
price_aprox_usd    14166 non-null float64
dtypes: float64(4), object(1)
memory usage: 553.4+ KB


In [8]:
set_datos = set_datos.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232005 entries, 0 to 232004
Data columns (total 5 columns):
property_type      232005 non-null object
lat                232005 non-null float64
lon                232005 non-null float64
superficie         232005 non-null float64
price_aprox_usd    232005 non-null float64
dtypes: float64(4), object(1)
memory usage: 8.9+ MB


### Agrupo por tipo de propiedad y luego por superficie, serian los lideres de knn

In [9]:
def redondear_superficie(superficie):
    """Convierto la superficie a un multiplo de 10"""
    superficie /= 10
    superficie = round(superficie) #lo hago entero
    superficie *= 10
    return int(superficie)

In [10]:
set_datos.loc[:,'superficie_actualizada'] = set_datos.loc[:,'superficie'].apply(redondear_superficie)
set_datos.head(2)

,property_type,lat,lon,superficie,price_aprox_usd,superficie_actualizada
0,house,-34.490998,-58.76748,300.0,110000.0,300
1,apartment,-34.587785,-58.38658,162.0,645000.0,160


In [19]:
set_datos_agrupados = set_datos.groupby('property_type')

In [1]:
minimo_datos_en_hoja = 2000

In [20]:
#Agrupo por superficie cada grupo
grupos = {}
for grupo in set_datos_agrupados.__iter__():
    clave = grupo[0]
    print(clave)
    valor = grupo[1].groupby('superficie_actualizada')
    diccionario = {}
    for group in valor.__iter__():
        sub_clave = group[0]
        dataframe = group[1]
        datos = Arbol_n_dimensiones(dataframe, ['lat','lon'], minimo_datos_en_hoja)
        diccionario[sub_clave] = datos
    grupos[clave] = diccionario

PH
0
0
apartment
1
0
3
2
4
4
3
2
1
2
4
4
3
3
2
1
0
5
4
5
5
4
3
2
4
3
5
4
5
4
3
2
1
3
4
3
2
5
4
5
5
4
3
5
4
4
3
2
1
0
4
4
3
2
3
4
4
3
2
1
4
3
2
4
4
3
3
2
1
0
4
3
2
3
4
4
3
2
1
4
3
2
4
4
3
3
2
1
0
3
2
3
2
1
2
3
2
1
0
3
2
3
2
1
2
2
1
0
3
2
1
2
1
0
2
2
1
1
0
1
0
1
0
0
1
0
0
house
0
0
0
0
1
1
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
store
0


In [21]:
set_datos_agrupados = grupos

#### Los datos quedan dic = {prop_type: {sup : arbol n dimensiones}}

## Algoritmo de KNN

In [23]:
def calcular_distancias_para_knn(datos,prueba):
    grupo = datos[prueba.property_type]
    prueba_sup = redondear_superficie(prueba.superficie)
    analizados = []
    for sup in range(prueba_sup - 10, prueba_sup + 20, 10):  #Miro sup +- 10
        try:
            a_analizar = grupo[sup].obtener_dataframe(prueba) #Lo busco en el arbol
            for i in range(len(a_analizar)):
                distancia = calcular_distancia(a_analizar.lat[i],a_analizar.lon[i],prueba.lat,prueba.lon)
                analizados.append((distancia,a_analizar.price_aprox_usd[i]))
        except:
            #Si no hay ninguna propiedad con superficie sup
            continue
    return analizados

In [24]:
def knn(datos,pruebas,k):
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.iloc[i])
        resultado = obtener_promedio_k_minimos(distancias,k)
        pruebas.resultado[i] = resultado
        pruebas.cantidad_analizados[i] = len(distancias)
    return pruebas

In [25]:
def knn_con_precision(datos,pruebas,lista_k): #Para buscar el mejor K de una lista
    resultados = {}
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.iloc[i])
        resultados = obtener_promedio_k_minimos(resultados,pruebas.price_aprox_usd[i],distancias,lista_k)
    return resultados

In [31]:
def obtener_promedio_k_minimos(resultados,real,distancias,lista_k):
    distancias.sort()
    for k in lista_k:
        k2 = min(k,len(distancias))
        a_analizar = distancias[:k2]
        suma = 0
        total = 0
        for x in range(len(a_analizar)):
            suma += a_analizar[x][1]
            total += 1
        if total != 0:
            res = suma / total
        else:
            res = 0
        lista = resultados.get(k,[])
        lista.append((real,res))
        resultados[k] = lista
    return resultados

In [27]:
def calcular_precision(resultados):
    res = {}
    for k in resultados.keys():
        suma = 0
        total = 0
        lista = resultados[k]
        for tupla in lista:
            suma += calcular_resultado(tupla[0],tupla[1])
            total += 1
        res[k] = round(suma*100 / total,2)
    return res

In [28]:
def calcular_resultado(real,calculado):
    porcentaje = 0.025
    if abs(real - calculado) <= real * porcentaje:
        return 1
    return 0

## Busqueda del hiperparametro K

##### Empezamos con algunos k salteados

In [33]:
lista_k = [5,10,20,30,50,100,200,350,500,750,1000]

inicio = time.strftime("%H:%M:%S")
pruebas = knn_con_precision(set_datos_agrupados,set_pruebas,lista_k)
precision = calcular_precision(pruebas)
print("Inicio: ({}) Final: ({})\n".format(inicio,time.strftime("%H:%M:%S")))
for k in lista_k:
    print("Para k = {} la precision es {}%".format(k,precision[k]))


Inicio: (16:07:18) Final: (16:07:24)

Para k = 5 la precision es 20.0%
Para k = 10 la precision es 8.0%
Para k = 20 la precision es 0.0%
Para k = 30 la precision es 4.0%
Para k = 50 la precision es 0.0%
Para k = 100 la precision es 0.0%
Para k = 200 la precision es 0.0%
Para k = 350 la precision es 4.0%
Para k = 500 la precision es 4.0%
Para k = 750 la precision es 4.0%
Para k = 1000 la precision es 0.0%
